# `QSDsan` Workshop Interactive Module <a class="anchor" id="top"></a>

- **Prepared by:**
    
    - [Yalin Li](https://qsdsan.readthedocs.io/en/latest/CONTRIBUTING.html)

- **Covered topics:**

    - [0. Instructions](#s0)
    - [1. Systems, TEA, LCA, and MCDA](#s1)
    
        - [1.1. System set up](#s1.1)
        - [1.2. TEA and LCA](#s1.2)
        - [1.3. MCDA](#s1.3)
    
    - [2. Uncertainty and Sensitivity Analyses](#s2)
    - [3. Country-Specific Analysis](#s3)

## 0. Instructions <a class="anchor" id="s0"></a>
Detailed instructions on how to use Jupyter Notebook can be found [here](https://realpython.com/jupyter-notebook-introduction/) (there are many online, this is just one example).

The key things to know about is that you can run a cell using `shift`/`ctrl`/`cmd`+`enter` or the `▶`/`▶Run` button on the menu bar as below.
<img src='files/run.png' alt='run'/>

Each of the three sections has multiple code cells. After you run all the cells, you will see prompts in the last cell and can play with it. You can use the "Back to top" link to go back to the top and select another section.

Remember that everything marked with "A" (e.g., `sysA`) is related to the pit latrine system and "B" is related to the urine-diverting dry toilet (UDDT) system.

Note that you need to install all the packages in "requirements.txt" (or clone the respective repositories) prior to running this notebook.

Have fun!

[Back to top](#top)

In [1]:
# This cell is just to show the version of the packages for result consistency
import qsdsan as qs, exposan as es
print(f'This tutorial was made with qsdsan v{qs.__version__}, exposan v{es.__version__}.')

This tutorial was made with qsdsan v1.3.1, exposan v1.3.2.


## 1. Systems, TEA, LCA, and MCDA <a class="anchor" id="s1"></a>

### 1.1. System set up <a class="anchor" id="s1.1"></a>

In [2]:
from ipywidgets import widgets as w
from systems import create_system

# Create systems
sysA = create_system('A')
sysB = create_system('B')

# Placeholders for outputs
diagram_out = w.Output()

# Display the system diagrams
def get_img_widget(file_path):
    file = open(file_path, 'rb')
    widget = w.Image(value=file.read(), format='png')
    return widget
diagramA_lbl = w.Label('Diagram for sysA:')
diagramB_lbl = w.Label('Diagram for sysB:')
diagramA =  get_img_widget('files/sysA.png')
diagramB =  get_img_widget('files/sysB.png')

@diagram_out.capture(clear_output=True, wait=True)
def display_diagrams():
    display(diagramA_lbl)
    display(diagramA)
    display(diagramB_lbl)
    display(diagramB)

##### Compiled run function #####
def run_system_interactive():
    display_diagrams()
    display(diagram_out)

In [3]:
run_system_interactive()

Output()

[Back to top](#top)

### 1.2. TEA and LCA <a class="anchor" id="s1.2"></a>

In [4]:
from ipywidgets import widgets as w
from systems import plot_tea_lca

# Placeholders for outputs
tea_lca_input = w.Output()
divider_out = w.Output()
result_out = w.Output()

##### User inputs #####
opt_lbl = w.Label('Which TEA/LCA metric(s) are you interested in (one or more)?')

tea_opts = ['net', 'CAPEX', 'OPEX', 'sales']
tea_box = w.SelectMultiple(
    options=tea_opts,
    value=['net'],
    rows=len(tea_opts),
    description='TEA metrics',
    disabled=False,
    style={'description_width': 'initial'},
    layout={'width':'50%'}
)

lca_opts = ['net', 'construction', 'operating', 'transportation', 'direct', 'offset']
lca_box = w.SelectMultiple(
    options=lca_opts,
    value=['net'],
    rows=len(lca_opts),
    description='LCA metrics',
    disabled=False,
    style=tea_box.style,
    layout=tea_box.layout
)

##### Divider #####
divider = w.Image(value=open('files/divider.png', 'rb').read(), format='png')

##### Results #####
@result_out.capture(clear_output=True, wait=True)
def simulate(btn=None):
    tea_metrics = tea_box.value
    lca_metrics = lca_box.value
    tea_msg = '", "'.join(tea_metrics)
    tea_msg = '"' + tea_msg + '"' 
    lca_msg = '", "'.join(lca_metrics)
    lca_msg = '"' + lca_msg + '"'
    tea_lca_result_lbl.value = f'Showing results for TEA ({tea_msg}) and LCA ({lca_msg}) metrics:'
    fig = plot_tea_lca((sysA, sysB), tea_metrics, lca_metrics)
    display(fig)

# Simulate button
simulate_btn = w.Button(
    description=' Simulate',
    disabled=False,
    # 'success' (green), 'info' (blue), 'warning' (yellow), 'danger' (red), or '' (grey)
    button_style='success',
    tooltip='Simulate and show results',
    icon='play' # (FontAwesome names without the `fa-` prefix)
)
simulate_btn.on_click(simulate)

# Result display
tea_lca_result_lbl = w.Label('')

##### Compiled run function #####
def run_tea_lca_interactive():
    with tea_lca_input:
        display(opt_lbl)
        display(tea_box)
        display(lca_box)
        display(simulate_btn)
    display(tea_lca_input)
    
    with divider_out:
        display(divider)
        display(tea_lca_result_lbl)
    display(divider_out)
    
    display(result_out)

In [5]:
run_tea_lca_interactive()

Output()

Output()

Output()

[Back to top](#top)

### 1.3. MCDA <a class="anchor" id="s1.3"></a>

In [6]:
from ipywidgets import widgets as w
from systems import create_mcda, plot_mcda

mcda = create_mcda((sysA, sysB))

# Placeholders for outputs
mcda_choice = w.Output()
divider_out = w.Output()
result_out = w.Output()

##### User inputs #####
tea_lbl = w.Label('Which TEA metric do you want to use for the economic criterion?')
tea_opts = ['net', 'CAPEX', 'OPEX', 'sales']
tea_btn = w.RadioButtons(
    options=tea_opts, 
    index=0,
    disabled=False)

lca_lbl = w.Label('Which LCA metric do you want to use for the environmental criterion?')
lca_opts = ['net', 'construction', 'operating', 'transportation', 'direct', 'offset']
lca_btn = w.RadioButtons(
    options=lca_opts, 
    index=0,
    disabled=False)

##### Divider #####
divider = w.Image(value=open('files/divider.png', 'rb').read(), format='png')

##### Results #####
@result_out.capture(clear_output=True, wait=True)
def simulate(btn=None):
    tea_metric = tea_btn.value
    lca_metric = lca_btn.value
    result_lbl.value = f'Showing MCDA results for the TEA ("{tea_metric}") ' \
        f'and the LCA metric ("{lca_metric}"):'
    fig = plot_mcda(mcda, tea_metric=tea_metric, lca_metric=lca_metric)
    display(fig)
    
# Simulate button
simulate_btn = w.Button(
    description=' Simulate',
    disabled=False,
    button_style='success',
    tooltip='Simulate and show results',
    icon='play'
)
simulate_btn.on_click(simulate)

# Result display
result_lbl = w.Label('')

##### Compiled run function #####
def run_mcda_interactive():
    with mcda_choice:
        display(tea_lbl)
        display(tea_btn)
        display(lca_lbl)
        display(lca_btn)
        display(simulate_btn)
    display(mcda_choice)
    
    with divider_out:
        display(divider)
        display(result_lbl)
    display(divider_out)
    
    display(result_out)

In [7]:
run_mcda_interactive()

Output()

Output()

Output()

[Back to top](#top)

## 2. Uncertainty and Sensitivity Analyses <a class="anchor" id="s2"></a>

In [8]:
from ipywidgets import widgets as w
from qsdsan import stats as s
from models import create_model, run_uncertainties, get_param_metric

modelA = create_model('A')
modelB = create_model('B')

# Placeholders for outputs
ua_sa_choice = w.Output()
divider_out = w.Output()
simulate_out = w.Output()
plot_out = w.Output()
figure_out = w.Output()

##### User inputs #####
# Number of samples
N_lbl = w.Label('How many samples would you like to run (100 takes 1-2 min)?')
N_txt = w.IntText(
    value='100',
    placeholder='N of samples',
    description='Integer:',
    disabled=False
)

# Uncertainty results
metrics_dct = {m.name_with_units:m.name for m in modelA.metrics} # names are the same for modelA and modelB
metrics_with_units = list(metrics_dct.keys())
ua_lbl = w.Label('Which metric(s) to show for uncertainty analysis results (one or more)?')
ua_box = w.SelectMultiple(
    options=metrics_with_units,
    rows=len(metrics_with_units),
    disabled=False,
    style={'description_width': 'initial'},
    layout={'width':'50%'}
)

# Sensitivity
sa_lbl = w.Label('Which metric to show for sensitivity analysis results (select one)?')
sa_menu = w.Dropdown(
    options=metrics_with_units,
    disabled=False,
)

##### Divider #####
divider = w.Image(value=open('files/divider.png', 'rb').read(), format='png')

##### Results #####
# Simulate button
simulate_btn = w.Button(
    description=' Simulate',
    disabled=False,
    button_style='success',
    tooltip='Evaluate models',
    icon='play'
)

# Wait prompt
wait_lbl = w.Label('')

@simulate_out.capture(clear_output=True, wait=True)
def simulate(btn=None):
    wait_lbl.value = 'Sit tight while the models are running...'
    global modelA, modelB
    modelA, modelB = run_uncertainties(N=int(N_txt.value))
    wait_lbl.value = 'Simulation done!'
    display(wait_lbl)
    display(plot_out)
simulate_btn.on_click(simulate)

# Plot button
plot_btn = w.Button(
    description='Plot Results',
    disabled=False,
    button_style='success',
    tooltip='Plot results',
    icon='play'
)
@figure_out.capture(clear_output=True, wait=True)
def plot(btn=None):
    ua_metrics = [metrics_dct[m] for m in ua_box.value]
    ua_metricsA = [get_param_metric(i, modelA, 'metric') for i in ua_metrics]
    print('\nUncertainty analysis results for sysA:\n')
    ua_figA = s.plot_uncertainties(modelA, x_axis=ua_metricsA)[0]
    display(ua_figA)
    print('\nUncertainty analysis results for sysB:\n')
    ua_metricsB = [get_param_metric(i, modelB, 'metric') for i in ua_metrics]
    ua_figB = s.plot_uncertainties(modelB, x_axis=ua_metricsB)[0]
    display(ua_figB)
    
    print('\nSensitivity analysis results for sysA:\n')
    sa_metric = metrics_dct[sa_menu.value]
    sa_metricA = get_param_metric(sa_metric, modelA, 'metric')
    spearmanA = s.get_correlations(modelA, input_y=sa_metricA, kind='Spearman')[0]
    sa_figA = s.plot_correlations(spearmanA, top=10)[0]
    display(sa_figA)
    
    print('\nSensitivity analysis results for sysB:\n')
    sa_metricB = get_param_metric(sa_metric, modelB, 'metric')
    spearmanB = s.get_correlations(modelB, input_y=sa_metricB, kind='Spearman')[0]
    sa_figB = s.plot_correlations(spearmanB, top=10)[0]
    display(sa_figB)
plot_btn.on_click(plot)
    
##### Compiled run function #####
def run_uncertainty_sensitivity_interactive():
    with ua_sa_choice:
        display(N_lbl)
        display(N_txt)
    display(ua_sa_choice)
    
    with divider_out:
        display(divider)
        display(simulate_btn)
    display(divider_out)
    
    with simulate_out:
        display(wait_lbl)
    display(simulate_out)
        
    with plot_out:
        display(divider)
        display(ua_lbl)
        display(ua_box)
        display(sa_lbl)
        display(sa_menu)
        display(plot_btn)
    
    display(figure_out)

In [9]:
run_uncertainty_sensitivity_interactive()

Output()

Output()

Output()

Output()

[Back to top](#top)

## 3. Country-Specific Analysis <a class="anchor" id="s3"></a>

In [10]:
from ipywidgets import widgets as w
from country_specific import val_dct_cached, get_val_df, get_results, plot

modelA = create_model('A', country_specific=True)
modelB = create_model('B', country_specific=True)
models = modelA, modelB
mcda.systems = (modelA.system, modelB.system)

# Placeholders for outputs
cs_choice = w.Output()
data_out = w.Output()
result_out = w.Output()

def clear_data_result_outs(btn=None):
    data_out.clear_output()
    result_out.clear_output()

all_outs = (cs_choice, data_out, result_out)
def clear_all_outs(btn=None):
    for out in all_outs:
        out.clear_output()

##### User inputs #####
# When using database
style = {'description_width': 'initial'}
country_lbl = w.Label('Please enter a country name:')
country_txt = w.Text('')

# When using user-input values
input_lbl = w.Label('Please enter values for the following parameters:')
caloric_intake = w.FloatText(
    value='2130',
    placeholder='baseline 2130 [kcal/d]',
    description='Caloric intake:',
    style=style, disabled=False)

vegetable_protein = w.FloatText(
    value='40.29',
    placeholder='baseline 40.29 [g/d]',
    description='Vegetable protein intake:',
    style=style, disabled=False)

animal_protein = w.FloatText(
    value='12.39',
    placeholder='baseline 12.39 [g/d]',
    description='Animal protein intake:',
    style=style, disabled=False)

N_price = w.FloatText(
    value='1.507',
    placeholder='baseline 1.507 [USD/kg N]',
    description='N fertilizer price:',
    style=style, disabled=False)

P_price = w.FloatText(
    value='3.983',
    placeholder='baseline 3.983 [USD/kg P]',
    description='P fertilizer price:',
    style=style, disabled=False)

K_price = w.FloatText(
    value='1.333',
    placeholder='baseline 1.333 [USD/kg K]',
    description='K fertilizer price:',
    style=style, disabled=False)

food_waste_ratio = w.FloatText(
    value='0.02',
    placeholder='baseline 0.02 [fraction]',
    description='Food waste ratio:',
    style=style, disabled=False)

price_level_ratio = w.FloatText(
    value='1',
    placeholder='baseline 1',
    description='Price level ratio:',
    style=style, disabled=False)

income_tax = w.FloatText(
    value='0.3',
    placeholder='baseline 0.3 [fraction]',
    description='Income tax:',
    style=style, disabled=False)

all_inputs = (
    caloric_intake,
    vegetable_protein,
    animal_protein,
    N_price,
    P_price,
    K_price,
    food_waste_ratio,
    price_level_ratio,
    income_tax,
)

# Economic weight slider
econ_wt_lbl = w.Label('Please use the slider to set the economic weight in MCDA')
econ_wt_slider = w.FloatSlider(
    value=0.5,
    min=0.,
    max=1.,
    step=0.01,
    description='Weight: ',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

# Whether to use database data
choice_btn = w.RadioButtons(
    options=['Yes', 'No'], 
    description='Would you like to use database values?',
    style=style, disabled=False)

def update_choice(btn=None):
    choice = choice_btn.value
    clear_all_outs()
    if choice_btn.value == 'Yes':
        with cs_choice:
            display(country_lbl)
            display(country_txt)
            display(country_btn)
    else:
        with cs_choice:
            display(input_lbl)
            for i in all_inputs:
                display(i)
            display(confirm_btn)
choice_btn.observe(update_choice)

# Display based on the choice
error_lbl = w.Label('')
param_lbl = w.Label('')
def display_database_data(btn=None):
    clear_data_result_outs()
    country = country_txt.value
    val_df = get_val_df(country)
    if isinstance(val_df, str): # no info for the country
        error_lbl.value = f'No data for country "{country}", please retry with another country name.'
        update_simulate_btn(False)
        with data_out:
            display(error_lbl)
    else:
        update_simulate_btn(True)
        with data_out:
            param_lbl.value = f'Parameter values for "{country}":'
            display(param_lbl)
            display(val_df)
            display(econ_wt_lbl)
            display(econ_wt_slider)
            display(simulate_btn)
country_btn = w.Button(
    description=' Retrieve Data',
    disabled=False,
    button_style='success',
    tooltip='Run database values for the parameters',
    icon='play'
)
country_btn.on_click(display_database_data)
            
def display_customized_data(btn=None):
    clear_data_result_outs()
    with data_out:
        param_lbl.value = f'Customized parameter values:'
        display(param_lbl)
        
        country_txt.value = country = 'customized'
        val_dct = {}
        for i in all_inputs:
            val_dct[i.description[:-1]] = float(i.value)
        val_df = get_val_df(val_dct)
        display(val_df)
        display(econ_wt_lbl)
        display(econ_wt_slider)
        display(simulate_btn)
confirm_btn = w.Button(
    description=' Confirm Data',
    disabled=False,
    button_style='success',
    tooltip='Confirm the input data for simulation',
    icon='play'
)
confirm_btn.on_click(display_customized_data)

##### Divider #####
divider = w.Image(value=open('files/divider.png', 'rb').read(), format='png')

##### Results #####
# Simulate button
def update_simulate_btn(VALID):
    if VALID:
        simulate_btn.button_style = 'success'
        simulate_btn.description = ' Simulate'
        simulate_btn.icon = 'play'
    else:
        simulate_btn.button_style = 'danger'
        simulate_btn.description = ' Invalid Inputs'
        simulate_btn.icon = 'stop'

@result_out.capture(clear_output=True, wait=True)
def simulate(btn=None):
    if btn.button_style != 'success': return # do not simulate if inputs are not valid
    
    if choice_btn.value == 'Yes': # with database data
        country = country_txt.value
        results_dct = get_results(country, models=models)
    else: # with user input data
        country = 'customized'
        val_dct = {}
        for i in all_inputs:
            val_dct[i.description[:-1]] = float(i.value)
        # val_dct_cached[country] = val_dct
        try: results_dct = get_results(val_dct, models=models)
        except: # unsuccessful simulation
            print('Cannot simulate for the provided parameters, please update inputs.')
            return

    weight = econ_wt_slider.value
    ax = plot(results_dct, mcda=mcda, econ_weight=econ_wt_slider.value)
    with result_out:
        print(f'Results for "{country}", economic weight is {weight}:')
        display(ax.figure)

simulate_btn = w.Button(
    description=' Simulate',
    disabled=False,
    button_style='success',
    tooltip='Simulate and show results',
    icon='play'
)
simulate_btn.on_click(simulate)

# # Clear button, not needed, but looks cool
# clear_btn = w.Button(
#     description=' Clear Outputs',
#     disabled=False,
#     button_style='warning',
#     tooltip='Clear all outputs',
#     icon='eraser'
# )
# clear_btn.on_click(clear_data_results)

##### Compiled run function #####
def run_country_specific_interactive():
    display(choice_btn)
    update_choice() # initialize display
    display(cs_choice)
    display(data_out)
    display(result_out)

In [11]:
run_country_specific_interactive()

RadioButtons(description='Would you like to use database values?', options=('Yes', 'No'), style=DescriptionSty…

Output()

[Back to top](#top)